In [13]:
import boto3
from botocore.exceptions import ClientError
import json
import time
import pandas as pd
import numpy as np
import uuid

personalize = boto3.client("personalize")
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client("personalize-events")

#### Campaign 생성

Campaign은 배포된 Solution Version입니다. 실질적을 추천서비스를 제공하는 endpoint입니다.
가격은 처리율 즉 TPS에 따라 결정되며, 일반적으로 테스트시에는 1로 설정합니다.

배포에는 약 10여분 정도가 소요됩니다

In [2]:
# 배포할 Solution Version
solution_version_arn = ""

In [3]:
suffix = "20230208"
campaign_name = "osungmart-campaign-" + suffix

In [ ]:
try:
    create_campaign_resp = personalize.create_campaign(
        name = campaign_name,
        solutionVersionArn = solution_version_arn,
        minProvisionedTPS = 1
    )
    
    campaign_arn = create_campaign_resp['campaignArn']
    print(json.dumps(create_campaign_resp, indent=2))
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceAlreadyExistsException":
    print(f"dataset group: {dataset_group_name} arleady exist")
  else:
    raise


In [ ]:
# Campaign 생성 모니터링
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_resp = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_resp["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 추천
cleand data sample에서 user id를 추출해서 사용자에 따른 상품 추천을 테스트한다.

In [6]:
cleaned_insteractions = pd.read_csv('data/cleaned_user_interactions.csv')
cleaned_insteractions.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,533326659,1005014,1667260814,cart
1,565865924,1005115,1667261019,cart
2,549256216,1002542,1667261154,cart
3,549256216,1002542,1667261242,cart
4,522355747,4804056,1667261445,cart


In [12]:
cleaned_insteractions['EVENT_TYPE'].unique()

array(['cart', 'purchase'], dtype=object)

In [10]:
# 3 user id sampling
users = cleaned_insteractions.sample(n=3)['USER_ID'].to_list()
users

[595097773, 542168651, 515749479]

In [ ]:
#campaign_arn = ""

In [ ]:
# get Recommendation
get_recommendation_resp = personalize_runtime.get_recommendataions(
    campaignArn = campaign_arn,
    userId = str(users[0])
item_list = get_recommendation_resp['itemList']
print(item_list)

#### Real time events Simulation

In [ ]:
event_tracker_name = "Osung-Event-Tracker"
dataset_group_arn = "osungmart-dataset-group20230204"

In [ ]:
# event tracker 생성
resp = personalize.create_event_tracker(
    name = event_tracker_name,
    datasetGroupArn = dataset_group_arn
)
print(resp['eventTrackerArn'])
print(resp['trakingId'])
TRACKING_ID = resp['trackingId']
event_tracker_arn = resp['eventTrackerArn']

##### method for simulations

In [ ]:
session_dict = {}
def send_event(USER_ID, ITEM_ID, EVENT_TYPE ='cart'):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict['USER_ID'] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
    
    # Configure Properties
    event = {
        "itemId": str(ITEM_ID)
    }
    
    event_json = json.dumps(event)
    # Make a Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId = str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            "sentAt": int(time.time()),
            "eventType": EVENT_TYPE,
            "properties": event_json
        }]
    )
            
def get_new_recommendation_uers_real_time(user_id, item_id_list):
    # Send event
    for item_id in item_id_list:
        send_event(USER_ID=user_id, ITEM_ID=item_id)
        time.sleep(5)
    
    # get recommendation
    get_recommendation_resp = personalize.get_recommendation(
        compaignArn = compaign_arn,
        userId = str(user_id),
    )
    
    item_list = get_recommendation_resp['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    
    return recommendation_list
    